<a href="https://colab.research.google.com/github/Gail529/AES/blob/main/automated_essay_scoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
 
import re
import string
import warnings
warnings.filterwarnings("ignore")  
from gensim.models import Word2Vec
 
 
#essay preprocessing 
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
import string
import re
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

#training the neural network 
from tensorflow.keras.layers import Embedding
from keras.models import Sequential, load_model, model_from_config
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Lambda
from tensorflow.keras.layers import Flatten
import keras.backend as K

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
essay_data= pd.read_excel('/content/training_set_rel3.xls',usecols=['essay_id','essay','domain1_score'])
essay_data.head(2)

,essay_id,essay,domain1_score
0,1,"Dear local newspaper, I think effects computer...",8.0
1,2,"Dear @CAPS1 @CAPS2, I believe that using compu...",9.0


In [ ]:
x=essay_data['essay']
y=essay_data['domain1_score']
len(x) 

12978

In [ ]:
essay_1=x[0]
print(essay_1)
#clean a single essay
print(sent_tokenize(essay_1))


Dear local newspaper, I think effects computers have on people are great learning skills/affects because they give us time to chat with friends/new people, helps us learn about the globe(astronomy) and keeps us out of troble! Thing about! Dont you think so? How would you feel if your teenager is always on the phone with friends! Do you ever time to chat with your friends or buisness partner about things. Well now - there's a new way to chat the computer, theirs plenty of sites on the internet to do so: @ORGANIZATION1, @ORGANIZATION2, @CAPS1, facebook, myspace ect. Just think now while your setting up meeting with your boss on the computer, your teenager is having fun on the phone not rushing to get off cause you want to use it. How did you learn about other countrys/states outside of yours? Well I have by computer/internet, it's a new way to learn about what going on in our time! You might think your child spends a lot of time on the computer, but ask them so question about the economy

In [ ]:
def sent_tokenize(text):
    stripped_essay = text.strip()
    
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw_sentences = tokenizer.tokenize(stripped_essay)
    
    tokenized_sentences = []
    for raw_sentence in raw_sentences:
        clean_sentence = re.sub("[^a-zA-Z0-9]"," ", raw_sentence)
        words = clean_sentence.lower().split()
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
        tokenized_sentences.append(words)

    return tokenized_sentences


In [ ]:
cleaned_essays=[]
for i in range(0,len(x)):
    essay=x[i]
    cleaned_essays.append(sent_tokenize(essay))
    

In [ ]:
sentences=[]
for essay in x:
    sentences += sent_tokenize(essay)

In [ ]:
sentences[0]

In [ ]:
text_dim = 300
print("Training Word2Vec model...")
wordvec_model = Word2Vec(sentences, size=text_dim, window=5, min_count=3, workers=4, sg=1)
print("Word2Vec model created.")
print("%d unique words represented by %d dimensional vectors" % (len(wordvec_model.wv.vocab), text_dim))
wordvec_model.save('wordvec_model')
print("Word2Vec model saved.")

Training Word2Vec model...
Word2Vec model created.
13134 unique words represented by 300 dimensional vectors
Word2Vec model saved.


In [ ]:
def clean_essays(essay):
    clean_essay= re.sub("[^a-zA-Z]", " ",essay)
    words = essay.lower().split()
    stops = set(stopwords.words("english"))
    words = [w for w in words if not w in stops]
    return (words)

In [ ]:
cleaned_essays = []
for essay in x:
        cleaned_essays.append(clean_essays(essay))

In [ ]:
def create_average_vec(essay):
    average = np.zeros((text_dim,), dtype='float32')
    num_words = 0.
    for word in essay:
        if word in wordvec_model.wv.vocab:
            average = np.add(average, wordvec_model.wv[word])
            num_words += 1.
    if num_words != 0.:
        average = np.divide(average, num_words)
    return average

In [ ]:
cleaned_vec = np.zeros((len(x),text_dim), dtype="float32")  
for i in range(len(cleaned_essays)):
    cleaned_vec[i] = create_average_vec(cleaned_essays[i])

print("Word vectors for all essays in the training data set are of shape:", cleaned_vec.shape)


Word vectors for all essays in the training data set are of shape: (12978, 300)


In [ ]:
cleaned_essays[0]

In [ ]:
def get_model():
    model = Sequential()
    model.add(LSTM(300, dropout=0.4, recurrent_dropout=0.4, input_shape=[1, 300], return_sequences=True))
    model.add(LSTM(64, recurrent_dropout=0.4))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='relu'))
    model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mae'])
    model.summary()
    return model

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=26)
